<a href="https://colab.research.google.com/github/hucarlos08/Nerf-Geo/blob/main/PreNerf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NeRF: Representing Scenes as Neural Radiance Fields for View Synthesis

The idea proposed in the paper [**NeRF: Representing Scenes as Neural Radiance Fields for View Synthesis**](https://arxiv.org/abs/2003.08934) aims to generate new views of an object (or even better, of a scene) using deep learning and computer graphics concepts. To better understand the idea, we first need to understand how information is captured in an image ([Ref](https://www.cambridge.org/core/books/multiple-view-geometry-in-computer-vision/0B6F289C78B2B23F596CAA76D3D43F7A)).


## Introduction: The 3D Revolution and Its Neural Future

In a world where 3D models are becoming as ubiquitous as 2D images, understanding how we represent and interact with three-dimensional spaces is more critical than ever. From designing complex machinery in CAD software to simulating the behavior of structures under stress, or even immersing ourselves in virtual and augmented reality experiences, 3D representation is the backbone of countless engineering applications.

But traditional methods for representing 3D objects, like meshes and point clouds, have their limitations. They can be cumbersome, computationally expensive, and struggle to capture the nuances of complex shapes and scenes. This is where Neural Radiance Fields (NeRF), a groundbreaking approach leveraging the power of neural networks, steps in.

NeRF has revolutionized the way we think about 3D representation, offering a more flexible, efficient, and photorealistic way to capture and recreate the world around us. In this notebook, we'll embark on a journey to understand the core concepts that underpin this exciting technology. We'll start with the fundamentals of 3D representation, explore how cameras capture light, delve into the magic of rendering, and get a taste of the incredible capabilities of neural networks. By the end, you'll have a solid grasp of the key ideas that make NeRF tick, setting the stage for deeper exploration into its technical intricacies.

So, buckle up and get ready to dive into the fascinating world of 3D representation and its neural future!

**In this notebook, we'll cover the following:**

1. **Representing the 3D World:** Exploring different ways to represent objects in 3D.
2. **Capturing Light: The Pinhole Camera Model:** Understanding how cameras capture 3D scenes.
3. **Generating Images: Rendering:** Learning how to create images from 3D data.
4. **Neural Networks: Our Universal Tool:** Discovering the power of neural networks for complex tasks.
5. **View Synthesis: The NeRF Challenge:**  Understanding the problem NeRF solves and how it differs from traditional methods.
6. **NeRF Sneak Peek:** A glimpse into the inner workings and incredible results of NeRF.




### 1. Representing the 3D World: Beyond Lines and Dots

In the digital realm, representing three-dimensional objects accurately and efficiently is a fundamental challenge. Engineering applications, from CAD modeling to virtual reality simulations, rely heavily on our ability to capture and manipulate 3D data. Let's explore some common ways to represent 3D objects and their trade-offs:

**1.1 Meshes: Connecting the Dots**

* **The Concept:** Imagine wrapping a wireframe around an object, forming a network of interconnected triangles. Each triangle is defined by three points called vertices, and the entire collection of triangles forms a mesh.
* **Applications:** Meshes are widely used in computer graphics and engineering due to their flexibility and ability to represent complex shapes.
* **Limitations:**
    * **Sharpness:** Meshes struggle to represent sharp edges or fine details accurately, requiring many tiny triangles.
    * **Complexity:** Complex models can have millions of triangles, making them computationally expensive to manipulate.

In [9]:
import numpy as np
import plotly.graph_objects as go
import meshio
from plotly.subplots import make_subplots

!wget https://raw.githubusercontent.com/empet/Datasets/master/hand.off;
mymesh = meshio.read('Nerf-Geo/data/hand.off') # https://raw.githubusercontent.com/empet/Datasets/master/hand.off
verts = mymesh.points
faces = mymesh.cells_dict['triangle']
x, y, z  = verts.T
i, j, k = faces.T

#plot surface triangulation
tri_vertices = verts[faces]
Xe = []
Ye = []
Ze = []
for T in tri_vertices:
    Xe += [T[k%3][0] for k in range(4)] + [ None]
    Ye += [T[k%3][1] for k in range(4)] + [ None]
    Ze += [T[k%3][2] for k in range(4)] + [ None]
       

fig = make_subplots(
          rows=1, cols=1, 
          subplot_titles=('3d Mesh', 'Mesh3d with vertex intensities', 'Mesh3d with cell intensities'),
          horizontal_spacing=0.02,
          specs=[[{"type": "scene"}]])  

fig.add_trace(go.Scatter3d(x=Xe,
                     y=Ye,
                     z=Ze,
                     mode='lines',
                     name='',
                     line=dict(color= 'rgb(40,40,40)', width=0.5)), 1, 1)                      
                    
                        
fig.update_layout(width=600, height=600, font_size=10)
fig.update_scenes(camera_eye_x=1.45, camera_eye_y=1.45, camera_eye_z=1.45)

--2024-05-15 01:58:26--  https://raw.githubusercontent.com/empet/Datasets/master/hand.off
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87592 (86K) [text/plain]
Saving to: ‘hand.off.2’

hand.off.2          100%[===================>]  85.54K  --.-KB/s    in 0.001s  

2024-05-15 01:58:26 (62.8 MB/s) - ‘hand.off.2’ saved [87592/87592]



In [7]:
!wget https://raw.githubusercontent.com/empet/Datasets/master/hand.off

--2024-05-15 01:57:08--  https://raw.githubusercontent.com/empet/Datasets/master/hand.off
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87592 (86K) [text/plain]
Saving to: ‘hand.off’

hand.off            100%[===================>]  85.54K  --.-KB/s    in 0.01s   

2024-05-15 01:57:08 (5.77 MB/s) - ‘hand.off’ saved [87592/87592]



## Point Clouds: A Scattered Representation**

[Image of a 3D point cloud model]

* **The Concept:** A point cloud is simply a collection of points in 3D space, each with coordinates (x, y, z). Imagine a swarm of fireflies representing the surface of an object.
* **Applications:** Point clouds are often generated from LiDAR (Light Detection and Ranging) scans, which measure distances to objects using lasers.
* **Limitations:**
    * **No Connectivity:** Point clouds lack information about how points are connected, making it difficult to distinguish the object's surface from empty space.
    * **Rendering:** Creating realistic images from point clouds is challenging due to the lack of surface information

In [ ]:
import open3d as o3d

#draw a point cloud with default parameter
dataset = o3d.data.PLYPointCloud()
office = o3d.io.read_point_cloud(dataset.path)
o3d.visualization.draw_plotly([office])

## Volumetric Representation: Filling the Space

* **The Concept:**  Imagine dividing 3D space into tiny cubes called voxels (like 3D pixels). Each voxel stores information about the density and color of the material it represents.
* **Advantages:**
    * **Continuous:** Volumetric representations capture the continuous nature of real-world objects, unlike discrete meshes or points.
    * **Implicit Surfaces:**  We can define surfaces implicitly as regions where density crosses a certain threshold.
* **Relevance to NeRF:** Neural Radiance Fields leverage this volumetric representation, learning the density and color of each point in space. This allows for incredibly detailed and realistic 3D scenes.

In [ ]:
# Import data
import time
import numpy as np

from skimage import io

vol = io.imread("https://s3.amazonaws.com/assets.datacamp.com/blog_assets/attention-mri.tif")
volume = vol.T
r, c = volume[0].shape

# Define frames
import plotly.graph_objects as go
nb_frames = 68

fig = go.Figure(frames=[go.Frame(data=go.Surface(
    z=(6.7 - k * 0.1) * np.ones((r, c)),
    surfacecolor=np.flipud(volume[67 - k]),
    cmin=0, cmax=200
    ),
    name=str(k) # you need to name the frame for the animation to behave properly
    )
    for k in range(nb_frames)])

# Add data to be displayed before animation starts
fig.add_trace(go.Surface(
    z=6.7 * np.ones((r, c)),
    surfacecolor=np.flipud(volume[67]),
    colorscale='Gray',
    cmin=0, cmax=200,
    colorbar=dict(thickness=20, ticklen=4)
    ))


def frame_args(duration):
    return {
            "frame": {"duration": duration},
            "mode": "immediate",
            "fromcurrent": True,
            "transition": {"duration": duration, "easing": "linear"},
        }

sliders = [
            {
                "pad": {"b": 10, "t": 60},
                "len": 0.9,
                "x": 0.1,
                "y": 0,
                "steps": [
                    {
                        "args": [[f.name], frame_args(0)],
                        "label": str(k),
                        "method": "animate",
                    }
                    for k, f in enumerate(fig.frames)
                ],
            }
        ]

# Layout
fig.update_layout(
         title='Slices in volumetric data',
         width=600,
         height=600,
         scene=dict(
                    zaxis=dict(range=[-0.1, 6.8], autorange=False),
                    aspectratio=dict(x=1, y=1, z=1),
                    ),
         updatemenus = [
            {
                "buttons": [
                    {
                        "args": [None, frame_args(50)],
                        "label": "&#9654;", # play symbol
                        "method": "animate",
                    },
                    {
                        "args": [[None], frame_args(0)],
                        "label": "&#9724;", # pause symbol
                        "method": "animate",
                    },
                ],
                "direction": "left",
                "pad": {"r": 10, "t": 70},
                "type": "buttons",
                "x": 0.1,
                "y": 0,
            }
         ],
         sliders=sliders
)

fig.show()

## Camera Model
We can see three coordinate frames:
1. World
2. Camera
3. Image

The idea is to understand how to perform transformations between each of the frames.

| ![2d-train](https://imgur.com/EvLTR2w.png) |
| :---: |
| **Figure 1**: Camera model. |


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Define camera parameters (you'll need to adjust these)
focal_length = 50  # In pixels
image_width = 640
image_height = 480

# Define camera pose (rotation and translation)
# Rotation matrix (Euler angles example, you might use quaternions or other representations)
theta_x = np.radians(30)  # Rotation around X axis
theta_y = np.radians(15)  # Rotation around Y axis
theta_z = np.radians(0)   # Rotation around Z axis

R_x = np.array([[1, 0, 0],
                [0, np.cos(theta_x), -np.sin(theta_x)],
                [0, np.sin(theta_x), np.cos(theta_x)]])
R_y = np.array([[np.cos(theta_y), 0, np.sin(theta_y)],
                [0, 1, 0],
                [-np.sin(theta_y), 0, np.cos(theta_y)]])
R_z = np.array([[np.cos(theta_z), -np.sin(theta_z), 0],
                [np.sin(theta_z), np.cos(theta_z), 0],
                [0, 0, 1]])
R = R_z @ R_y @ R_x  # Combined rotation matrix

# Translation vector
T = np.array([-2, 1, 5])  # Camera position in the world

# Combine rotation and translation into a transformation matrix
P = np.hstack((R, T.reshape(-1, 1)))
P = np.vstack((P, [0, 0, 0, 1]))  # Homogeneous coordinates


# Define a point in the world coordinate frame
X_w = np.array([3, 2, 10, 1])  # Homogeneous coordinates (x, y, z, 1)

# Transform to camera coordinates
X_c = P @ X_w

# Project to image plane (assuming pinhole camera model)
x = focal_length * X_c[0] / X_c[2] + image_width / 2
y = focal_length * X_c[1] / X_c[2] + image_height / 2

print("World coordinates:", X_w[:3])
print("Camera coordinates:", X_c[:3])
print("Projected image coordinates (pixels):", x, y)


## From World to Camera

### World Coordinates
We usually represent the **world** coordinates using a vector $\mathbf{X}_w\in\mathbb{R}^3$ defined as:
$$
\mathbf{X}_w=\begin{bmatrix}
x_w \\
y_w \\
z_w
\end{bmatrix}
$$.

### Camera Coordinates
This is a dynamic reference frame that moves as we move the camera to take pictures. If a point $P$ has coordinates $\mathbf{X}_w$ in the *world frame*, in the **camera frame** we assign the coordinates:
$$
\mathbf{X}_c=\begin{bmatrix}
x_c \\
y_c \\
z_c
\end{bmatrix}
$$

**The location of point $P$ does not change. Only the way we look at the point changes with the change in the camera's reference frame.**

### Coordinate Transformation
It is possible to transform the coordinates of point $P$ from world to camera using the following equation:
$$
\mathbf{X}_c=R\times(\mathbf{X}_w-\mathbf{C}_w),
$$
where:
1. $R$ is a matrix representing the **orientation** of the camera with respect to the *world* coordinates.
2. $\mathbf{C}_w$ is a vector representing the **position** of the camera with respect to the *world* coordinates.

The matrix $C_{ext}$ is known as **extrinsic parameter matrix** because it represents the rotation and translation values, which are external properties of the camera.


## Image Formation

Let's return to the reference frames of the camera and the image

| ![2d-train](https://imgur.com/GCMmOLa.png) |
| :---: |
| **Figure 1**: Modelo proyectivo. |


### Projection to Image Plane
The projection to the image plane is done using the intrinsic parameters of the camera. If a point $P$ has coordinates $\mathbf{X}_c$ in the camera frame, its projection onto the image plane is given by:
$$
\begin{align}
x_i=f\frac{x_c}{z_c} \\
y_i=f\frac{y_c}{z_c}
\end{align}
$$
It is common for the image origin to be the top-left corner, so it is convenient to add offsets with respect to the image plane center:
$$
\begin{align}
x_i=f\frac{x_c}{z_c}+\delta_x \\
y_i=f\frac{y_c}{z_c}+\delta_y
\end{align}
$$
We can rewrite the previous equations as:
\begin{equation}
\begin{bmatrix}
fx_c+ z_c\delta_x\\
fy_c+ z_c\delta_y\\
~z_c\\
\end{bmatrix}=
\begin{bmatrix}
f_x&0&\delta_x&0\\
0&f_y&\delta_y&0\\
0&0&1&0\\
\end{bmatrix} \begin{bmatrix}
x_{c}\\
y_{c}\\
z_{c}\\
1\\
\end{bmatrix}
\tag{3}
\end{equation}
The first matrix on the right-hand side of the equation is known as the intrinsic parameters matrix $K$. It is termed intrinsic because it includes the focal length $f$ and the image plane center, which are internal properties of the camera.


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d

if not os.path.exists('tiny_nerf_data.npz'):
  !wget http://cseweb.ucsd.edu/~viscomp/projects/LF/papers/ECCV20/nerf/tiny_nerf_data.npz

data = np.load('tiny_nerf_data.npz')
images = data['images']
poses = data['poses']
focal = data['focal']

print(f'Images shape: {images.shape}')
print(f'Poses shape: {poses.shape}')
print(f'Focal length: {focal}')

height, width = images.shape[1:3]

testimg_idx = 3
testimg, testpose = images[testimg_idx], poses[testimg_idx]

plt.imshow(testimg)
print('Pose')
print(testpose)


In [ ]:
dirs = np.stack([np.sum([0, 0, -1] * pose[:3, :3], axis=-1) for pose in poses])
origins = poses[:, :3, -1]

ax = plt.figure(figsize=(12, 8)).add_subplot(projection='3d')
_  = ax.quiver(
  origins[..., 0].flatten(),
  origins[..., 1].flatten(),
  origins[..., 2].flatten(),
  dirs[..., 0].flatten(),
  dirs[..., 1].flatten(),
  dirs[..., 2].flatten(), length=0.5, normalize=True)
plt.show()
